# Search and Filter DataFrames in PySpark HW

Now it's time to put what you've learn into action with a homework assignment!

In case you need it again, here is the link to the documentation for the full list available function in pyspark.sql.functions library:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions


### First set up your Spark Session!
Alright so first things first, let's start up our pyspark instance.

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark-practice").getOrCreate()




22/01/17 17:57:32 WARN Utils: Your hostname, Legion resolves to a loopback address: 127.0.1.1; using 172.27.192.232 instead (on interface eth0)
22/01/17 17:57:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/01/17 17:57:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
spark

core = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("Number of cores present:",core)

Number of cores present: 1


## Read in the DataFrame for this Notebook

We will be continuing to use the fifa19.csv file for this notebook. Make sure that you are writting the correct path to the file. 

In [7]:
path = "/home/iron/Documents/1.Learning/2. PYSPARK_BASICS/Read Write Validate Datasets/datasets/"

fifa = spark.read.csv(path+"fifa19.csv", inferSchema=True, header=True)
# fifa.describe()

DataFrame[summary: string, ID: string, Name: string, Age: string, Photo: string, Nationality: string, Flag: string, Overall: string, Potential: string, Club: string, Club Logo: string, Value: string, Wage: string, Special: string, Preferred Foot: string, International Reputation: string, Weak Foot: string, Skill Moves: string, Work Rate: string, Body Type: string, Real Face: string, Position: string, Jersey Number: string, Joined: string, Loaned From: string, Contract Valid Until: string, Height: string, Weight: string, LS: string, ST: string, RS: string, LW: string, LF: string, CF: string, RF: string, RW: string, LAM: string, CAM: string, RAM: string, LM: string, LCM: string, CM: string, RCM: string, RM: string, LWB: string, LDM: string, CDM: string, RDM: string, RWB: string, LB: string, LCB: string, CB: string, RCB: string, RB: string, Crossing: string, Finishing: string, HeadingAccuracy: string, ShortPassing: string, Volleys: string, Dribbling: string, Curve: string, FKAccuracy: str

## About this dataframe

The **fifa19.csv** dataset includes a list of all the FIFA 2019 players and their attributes listed below: 

 - **General**: Age, Nationality, Overall, Potential, Club
 - **Metrics:** Value, Wage
 - **Player Descriptive:** Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight
 - **Possition:** LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, 
 - **Other:** Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

**Source:** https://www.kaggle.com/karangadiya/fifa19

Use the .toPandas() method to view the first few lines of the dataset so we know what we are working with. 

In [8]:
fifa.limit(5).toPandas()

22/01/17 18:01:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,96,33,28,26,6,11,15,14,8,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,95,28,31,23,7,11,15,14,11,€127.1M
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,...,94,27,24,33,9,9,15,15,11,€228.1M
3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,...,68,15,21,13,90,85,87,88,94,€138.6M
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,...,88,68,58,51,15,13,5,10,13,€196.4M


Now print the schema of the dataset so we can see the data types of all the varaibles. 

In [9]:
fifa.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Photo: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- Overall: integer (nullable = true)
 |-- Potential: integer (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club Logo: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- Wage: string (nullable = true)
 |-- Special: integer (nullable = true)
 |-- Preferred Foot: string (nullable = true)
 |-- International Reputation: integer (nullable = true)
 |-- Weak Foot: integer (nullable = true)
 |-- Skill Moves: integer (nullable = true)
 |-- Work Rate: string (nullable = true)
 |-- Body Type: string (nullable = true)
 |-- Real Face: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Jersey Number: integer (nullable = true)
 |-- Joined: string (nullable = true)
 |-- Loaned From: string (nullable = true)
 |-- Contract Valid U

## Now let's get started!

### First things first..... import the pyspark sql functions library

Since we know we will be using it a lot.

In [21]:
from pyspark.sql.functions import *


### 1. Select the Name and Position of each player in the dataframe

In [11]:
fifa.select("Name", "Position").show()

+-----------------+--------+
|             Name|Position|
+-----------------+--------+
|         L. Messi|      RF|
|Cristiano Ronaldo|      ST|
|        Neymar Jr|      LW|
|           De Gea|      GK|
|     K. De Bruyne|     RCM|
|        E. Hazard|      LF|
|        L. Modrić|     RCM|
|        L. Suárez|      RS|
|     Sergio Ramos|     RCB|
|         J. Oblak|      GK|
|   R. Lewandowski|      ST|
|         T. Kroos|     LCM|
|         D. Godín|      CB|
|      David Silva|     LCM|
|         N. Kanté|     LDM|
|        P. Dybala|      LF|
|          H. Kane|      ST|
|     A. Griezmann|     CAM|
|    M. ter Stegen|      GK|
|      T. Courtois|      GK|
+-----------------+--------+
only showing top 20 rows



### 1.1 Display the same results from above sorted by the players names

In [13]:
fifa.select("Name", "Position").orderBy("Name").alias("Name").show()

+--------------+--------+
|          Name|Position|
+--------------+--------+
|      A. Abang|      ST|
| A. Abdellaoui|      LB|
|  A. Abdennour|      CB|
|       A. Abdi|      CM|
| A. Abdu Jaber|      ST|
|A. Abdulhameed|      GK|
|  A. Abedzadeh|      GK|
|      A. Abeid|      LB|
|      A. Ablet|     LWB|
|    A. Abrashi|     CDM|
|   A. Abruscia|    null|
|    A. Absalem|      LB|
|    A. Accardi|      CB|
|    A. Acevedo|     RCB|
|     A. Acosta|      RM|
|     A. Acosta|      LB|
|     A. Acquah|     LCM|
|       A. Adam|      ST|
|      A. Addai|      ST|
|      A. Ademi|     RDM|
+--------------+--------+
only showing top 20 rows



### 2. Select only the players who belong to a club begining with FC

In [17]:
# fifa.limit(5).toPandas()

fifa.select("Name","Club").where(fifa.Club.like("%FC%")).show()

+---------------+--------------------+
|           Name|                Club|
+---------------+--------------------+
|       L. Messi|        FC Barcelona|
|      L. Suárez|        FC Barcelona|
| R. Lewandowski|   FC Bayern München|
|  M. ter Stegen|        FC Barcelona|
|Sergio Busquets|        FC Barcelona|
|       M. Neuer|   FC Bayern München|
|   J. Rodríguez|   FC Bayern München|
|       Coutinho|        FC Barcelona|
|     M. Hummels|   FC Bayern München|
|      S. Umtiti|        FC Barcelona|
|     Jordi Alba|        FC Barcelona|
|     I. Rakitić|        FC Barcelona|
|          Piqué|        FC Barcelona|
|      T. Müller|   FC Bayern München|
|         Thiago|   FC Bayern München|
|     J. Kimmich|   FC Bayern München|
|       D. Alaba|   FC Bayern München|
|       Paulinho|Guangzhou Evergra...|
|     Y. Brahimi|            FC Porto|
|     J. Boateng|   FC Bayern München|
+---------------+--------------------+
only showing top 20 rows



In [18]:
fifa.select("Name","Club").where(fifa.Club.startswith("FC")).show()

+---------------+-----------------+
|           Name|             Club|
+---------------+-----------------+
|       L. Messi|     FC Barcelona|
|      L. Suárez|     FC Barcelona|
| R. Lewandowski|FC Bayern München|
|  M. ter Stegen|     FC Barcelona|
|Sergio Busquets|     FC Barcelona|
|       M. Neuer|FC Bayern München|
|   J. Rodríguez|FC Bayern München|
|       Coutinho|     FC Barcelona|
|     M. Hummels|FC Bayern München|
|      S. Umtiti|     FC Barcelona|
|     Jordi Alba|     FC Barcelona|
|     I. Rakitić|     FC Barcelona|
|          Piqué|     FC Barcelona|
|      T. Müller|FC Bayern München|
|         Thiago|FC Bayern München|
|     J. Kimmich|FC Bayern München|
|       D. Alaba|FC Bayern München|
|     Y. Brahimi|         FC Porto|
|     J. Boateng|FC Bayern München|
|       A. Vidal|     FC Barcelona|
+---------------+-----------------+
only showing top 20 rows



### 3. Who is the oldest player in the dataset and how old are they?

Display only the name and age of the oldest player.

In [23]:
# fifa.printSchema()

fifa.select("Name","Club","Age").sort(desc("Age")).show()

+---------------+--------------------+---+
|           Name|                Club|Age|
+---------------+--------------------+---+
|       O. Pérez|             Pachuca| 45|
|      T. Warner|  Accrington Stanley| 44|
|  K. Pilkington|    Cambridge United| 44|
|    S. Narazaki|      Nagoya Grampus| 42|
|       B. Nivet|        ESTAC Troyes| 41|
|   H. Sulaimani|           Ohod Club| 41|
|       M. Tyler| Peterborough United| 41|
|      J. Villar|                null| 41|
|       C. Muñoz|CD Universidad de...| 41|
|   A. Al Basisi|           Ohod Club| 40|
|        W. Díaz| Jaguares de Córdoba| 40|
|    S. Nakamura|        Júbilo Iwata| 40|
|    B. Castillo|      Atlético Huila| 40|
|   C. Lucchetti|    Atlético Tucumán| 40|
|       F. Kippe|       Lillestrøm SK| 40|
|    A. Bizzarri|              Foggia| 40|
|    Y. Nakazawa| Yokohama F. Marinos| 40|
|      G. Buffon| Paris Saint-Germain| 40|
|P. van der Vlag|            FC Emmen| 40|
|         Hilton|     Montpellier HSC| 40|
+----------

### 4. Select only the following players from the dataframe:

 - L. Messi
 - Cristiano Ronaldo

In [25]:
fifa[fifa.Name.isin("L. Messi","Cristiano Ronaldo")].limit(4).toPandas()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,96,33,28,26,6,11,15,14,8,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,95,28,31,23,7,11,15,14,11,€127.1M


### 5. Can you select the first character from the Release Clause variable which indicates the currency used?

In [30]:
from pyspark.sql.functions import *

fifa.select("Release Clause", fifa["Release Clause"].substr(1,1).alias("Currency") ).show()

+--------------+--------+
|Release Clause|Currency|
+--------------+--------+
|       €226.5M|       €|
|       €127.1M|       €|
|       €228.1M|       €|
|       €138.6M|       €|
|       €196.4M|       €|
|       €172.1M|       €|
|       €137.4M|       €|
|         €164M|       €|
|       €104.6M|       €|
|       €144.5M|       €|
|       €127.1M|       €|
|       €156.8M|       €|
|        €90.2M|       €|
|         €111M|       €|
|       €121.3M|       €|
|       €153.5M|       €|
|       €160.7M|       €|
|       €165.8M|       €|
|       €123.3M|       €|
|       €113.7M|       €|
+--------------+--------+
only showing top 20 rows



### 6. Can you select only the players who are over the age of 40?

In [32]:
fifa.filter("Age>40").select("Name","Club","Age").show()

+-------------+--------------------+---+
|         Name|                Club|Age|
+-------------+--------------------+---+
|    J. Villar|                null| 41|
|     B. Nivet|        ESTAC Troyes| 41|
|     O. Pérez|             Pachuca| 45|
|     C. Muñoz|CD Universidad de...| 41|
|  S. Narazaki|      Nagoya Grampus| 42|
| H. Sulaimani|           Ohod Club| 41|
|     M. Tyler| Peterborough United| 41|
|    T. Warner|  Accrington Stanley| 44|
|K. Pilkington|    Cambridge United| 44|
+-------------+--------------------+---+



### That's is for now... Great Job!